In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('big_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6807 entries, 0 to 6806
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           6807 non-null   int64  
 1   INJURY_BAD           6807 non-null   float64
 2   HELMET_USED          6807 non-null   float64
 3   age_bins             6807 non-null   object 
 4   SAFE_PEDAL_ACTION    6807 non-null   object 
 5   PROTECTIVE_CLOTHING  6807 non-null   float64
 6   SAFE_LOCATION        6807 non-null   float64
 7   WEATHER_CAT          6807 non-null   object 
 8   road_surf_bins       6807 non-null   object 
 9   DAYTIME              6807 non-null   int64  
 10  SPEED_RATING         6807 non-null   object 
 11  physical_bins        6807 non-null   object 
 12  hour_bins            6807 non-null   object 
 13  SEX                  6807 non-null   object 
dtypes: float64(4), int64(2), object(8)
memory usage: 744.6+ KB


In [4]:
df = df.drop(['Unnamed: 0','road_surf_bins'],axis=1)

In [5]:
df = df.loc[df['SEX'] != 'X']

In [6]:
df = df.loc[df['physical_bins'] != 'Shaken']

In [7]:
df['SEX'].value_counts()

M    5202
F    1369
Name: SEX, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6571 entries, 0 to 6806
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INJURY_BAD           6571 non-null   float64
 1   HELMET_USED          6571 non-null   float64
 2   age_bins             6571 non-null   object 
 3   SAFE_PEDAL_ACTION    6571 non-null   object 
 4   PROTECTIVE_CLOTHING  6571 non-null   float64
 5   SAFE_LOCATION        6571 non-null   float64
 6   WEATHER_CAT          6571 non-null   object 
 7   DAYTIME              6571 non-null   int64  
 8   SPEED_RATING         6571 non-null   object 
 9   physical_bins        6571 non-null   object 
 10  hour_bins            6571 non-null   object 
 11  SEX                  6571 non-null   object 
dtypes: float64(4), int64(1), object(7)
memory usage: 667.4+ KB


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [10]:
y = df['INJURY_BAD']
X = df.drop('INJURY_BAD',axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=312)

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
ohe = OneHotEncoder()
X_train_encoded = ohe.fit_transform(X_train)
X_test_encoded = ohe.transform(X_test)

In [13]:
dt = DecisionTreeClassifier(max_depth=25,random_state=312)
dt.fit(X_train_encoded,y_train)

DecisionTreeClassifier(max_depth=25, random_state=312)

In [14]:
y_train_pred = dt.predict(X_train_encoded)
y_test_pred = dt.predict(X_test_encoded)

from sklearn.metrics import precision_score,recall_score

train_p = precision_score(y_train,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_train,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')

Train Precision: 0.9642857142857143
Test Precision: 0.05660377358490566
Train Recall: 0.27
Test Recall: 0.02127659574468085


In [15]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

#plt.figure(figsize=(16, 12))
#plot_tree(dt, filled=True)
#plt.show()

In [16]:
from imblearn.under_sampling import RandomUnderSampler

under = RandomUnderSampler(random_state=312)
X_u, y_u = under.fit_resample(X_train_encoded,y_train)

In [17]:
dt = DecisionTreeClassifier(max_depth=2,random_state=312)
dt.fit(X_u,y_u)

DecisionTreeClassifier(max_depth=2, random_state=312)

In [18]:
y_train_pred = dt.predict(X_u)
y_test_pred = dt.predict(X_test_encoded)

train_p = precision_score(y_u,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_u,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')

Train Precision: 0.5693430656934306
Test Precision: 0.1267605633802817
Train Recall: 0.624
Test Recall: 0.6382978723404256


In [19]:
#plt.figure(figsize=(16, 12))
#plot_tree(dt, filled=True)
#plt.show()

In [20]:
importances = list(dt.feature_importances_)

In [21]:
xu_df = pd.DataFrame(X_u.todense(),columns=ohe.get_feature_names())

AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names'

In [ ]:
names = list(xu_df.columns)

In [ ]:
importance_names = dict(zip(names,importances))

In [ ]:
importance_names

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train_df = pd.DataFrame(X_train_encoded.todense(),columns=ohe.get_feature_names())
X_test_df = pd.DataFrame(X_test_encoded.todense(),columns=ohe.get_feature_names())
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)

In [ ]:
log_model = LogisticRegression(class_weight='balanced',max_iter=1000,random_state=312)

In [ ]:
log_model.fit(X_train_scaled,y_train)

In [ ]:
y_train_pred = log_model.predict(X_train_scaled)
y_test_pred = log_model.predict(X_test_scaled)

train_p = precision_score(y_train,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_train,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200,max_depth=4,random_state=312)
clf.fit(X_u,y_u)

In [ ]:
y_train_pred = clf.predict(X_u)
y_test_pred = clf.predict(X_test_encoded)

train_p = precision_score(y_u,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_u,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')

In [ ]:
importances = list(clf.feature_importances_)
names = list(xu_df.columns)
importance_names = dict(zip(names,importances))
importance_names

In [ ]:
from sklearn.ensemble import VotingClassifier

avg = VotingClassifier(estimators=[('dt',dt),('clf',clf)])
avg.fit(X_u,y_u)

In [ ]:
y_train_pred = avg.predict(X_u)
y_test_pred = avg.predict(X_test_encoded)

train_p = precision_score(y_u,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_u,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

y_score = log_model.decision_function(X_test_scaled)
precision,recall,thresholds = precision_recall_curve(y_test,y_score)
print('AUC: {}'.format(auc(recall, precision)))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Plot a Precision-Recall Curve
sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

plt.figure(figsize=(10, 8))
lw = 2
plt.plot(recall, precision, color='darkorange',
         lw=lw, label='Precision-Recall curve (AUC = 0.20)')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(log_model,X_test_scaled,y_test);

In [ ]:
plot_confusion_matrix(clf,X_test_encoded,y_test);

In [ ]:
coef_dict = {}
for coef, feat in zip(log_model.coef_[0,:],X_train_df.columns):
    coef_dict[feat] = coef
for i,v in coef_dict.items():
    coef_dict[i] = round(v,2)
coef_dict

In [ ]:
import numpy as np
selected_five = np.array([-0.37,0.29,0.27,0.23,-0.18])
np.exp(selected_five)

In [ ]:
#from sklearn.model_selection import GridSearchCV

#param_grid = {
#    'C': [0.001, 0.01, 0.1, 1, 10, 100],
#    'penalty': ['l1', 'l2', 'elasticnet'],
#    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#}

#grid_search = GridSearchCV(log_model, param_grid, cv=5)
#grid_search.fit(X_train_scaled, y_train)
#
# Get the best hyperparameters and score
#best_params = grid_search.best_params_
#best_score = grid_search.best_score_

# Train a new Logistic Regression model with the best hyperparameters
#logreg_best = LogisticRegression(random_state=312, **best_params)
#logreg_best.fit(X_train_scaled, y_train)

# Obtain predictions on the test set
#y_pred = logreg_best.predict(X_test_scaled)

In [ ]:
# Generate a classification report for the predictions
#from sklearn.metrics import classification_report

#print(classification_report(y_test, y_pred))

In [ ]:
#best_params

In [ ]:
log_model = LogisticRegression(C=.001,class_weight='balanced',max_iter=1000,random_state=312)
log_model.fit(X_train_scaled,y_train)

y_train_pred = log_model.predict(X_train_scaled)
y_test_pred = log_model.predict(X_test_scaled)

train_p = precision_score(y_train,y_train_pred)
test_p = precision_score(y_test,y_test_pred)
train_r = recall_score(y_train,y_train_pred)
test_r = recall_score(y_test,y_test_pred)

print(f'Train Precision: {train_p}')
print(f'Test Precision: {test_p}')
print(f'Train Recall: {train_r}')
print(f'Test Recall: {test_r}')